In [1]:
import pandas as pd 
import yfinance as yf
from datetime import datetime, timedelta


In [2]:
SP500_Url = "https://en.wikipedia.org/wiki/Nasdaq-100"
data_table = pd.read_html(
    SP500_Url,
    storage_options={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                                   "(KHTML, like Gecko) Chrome/123.0 Safari/537.36"}
)
data_table

[                                     0                                      1
 0   Nasdaq-100 Index from 1985 to 2015     Nasdaq-100 Index from 1985 to 2015
 1                           Foundation         January 31, 1985; 40 years ago
 2                             Operator                           Nasdaq, Inc.
 3                            Exchanges                                 Nasdaq
 4                       Trading symbol                            ^NDX or NDX
 5                         Constituents                                    101
 6                                 Type                              Large-cap
 7                           Market cap  US$25.34 trillion (as of May 1, 2025)
 8                     Weighting method     Free-float capitalization-weighted
 9                      Related indices                   Nasdaq Financial-100
 10                             Website                  nasdaq.com/nasdaq-100,
    Category  All-Time Highs[8]         All-Time Hi

In [3]:
tickers = data_table[4]["Ticker"].tolist()

In [4]:
supp = ['ASML', 'AZN', 'F', 'G', 'GOOGL', 'H', 'I', 'J', 'L', 'O', 'Q', 'R', 'U', 'VRSK', 'W', 'Y']

tickers = [x for x in tickers if x not in supp]
print(len(tickers), len(supp))

97 16


In [5]:
# Extraction des données
Date = data_table[5]['Date']
Removed = data_table[5]['Removed']['Ticker']  # Supposant que 'Removed' est un DataFrame ou Series
Added = data_table[5]['Added']['Ticker']      # Supposant que 'Added' est un DataFrame ou Series

# Combinaison des données dans un DataFrame
Data_Change = pd.concat([Date, Added, Removed], axis = 1)

#Date au format Date
Data_Change['Date'] = pd.to_datetime(Data_Change['Date'])

# Changer le format de la date en 'jour/mois/année'
Data_Change['Date'] = Data_Change['Date'].dt.strftime('%Y-%m-%d')


# Renommer toutes les colonnes
Data_Change.columns = ['Date', 'Added', 'Removed']

# Mettre 'colonne_a' en tant qu'index
Data_Change.set_index('Date', inplace=True)

# Nan en vide
Data_Change = Data_Change.fillna('')

Data_Change

,Added,Removed
Date,,
2025-07-28,TRI,
2025-07-17,,ANSS
2025-05-19,SHOP,MDB
2024-12-23,PLTR,ILMN
2024-12-23,MSTR,MRNA
...,...,...
2007-07-12,FWLT,BMET
2007-06-01,CEPH,MEDI
2007-03-08,UAUA,AEOS


In [6]:
len(tickers)

97

In [7]:
# Supposons que la liste tickers ait déjà 503 éléments au départ
# tickers = ['AAPL', 'MSFT', ...]  # Exemples de tickers initiaux

# Initialiser un dictionnaire vide
dict = {}

# Boucle pour parcourir les données et mettre à jour le dictionnaire
for i in range(0, len(Data_Change)):
    
    # Ajouter les nouveaux tickers (si ce n'est pas déjà un ensemble vide)
    tickers.extend(Data_Change.Added[i])
    
    # Essayer de supprimer les tickers retirés, si possible
    try:
        tickers = [t for t in tickers if t not in Data_Change.Removed[i]]  # Supprimer les tickers retirés
    except ValueError:
        # Si l'élément à supprimer n'existe pas dans la liste, continuer sans erreur
        pass
    
    # Trier les tickers
    tickers = sorted(set(tickers))  # Utilisation de set pour éviter les doublons

    # Ajouter une **copie** de la liste `tickers` dans le dictionnaire
    dict[Data_Change.index[i]] = tickers.copy()

df = pd.DataFrame(list(dict.items()), columns=['date', 'tickers'])

df['date'] = pd.to_datetime(df['date'])

# Mettre la colonne 'date' comme index
df.set_index('date', inplace=True)

df

/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_4171/1527443372.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tickers.extend(Data_Change.Added[i])
/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_4171/1527443372.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tickers = [t for t in tickers if t not in Data_Change.Removed[i]]  # Supprimer les tickers retirés


,tickers
date,
2025-07-28,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."


In [8]:
# Récupère l'index en liste
new_index = list(df.index)

# Remplace la première valeur
new_index[0] = datetime.today().date()

# Réapplique l'index modifié
df.index = pd.to_datetime(new_index)

In [9]:
df

,tickers
2025-10-01,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2007-02-14,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."


In [10]:

data_SP500_old = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet')
last_date = data_SP500_old['Date'].sort_values(ascending=True).iloc[-1] + timedelta(days=1)
last_date


Timestamp('2025-09-27 00:00:00')

In [11]:

df['Last_Date'] = last_date
df

,tickers,Last_Date
2025-10-01,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-27
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-27
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-27
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-27
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-27
...,...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-27
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-27
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-27
2007-02-14,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-27


In [12]:

# Initialiser une DataFrame vide pour stocker toutes les données
data_SP500 = pd.DataFrame()

df = df.explode('tickers')
end_date = df[df.index > last_date].groupby('tickers').apply(lambda x: x.index.max())

# Combiner les résultats dans une nouvelle DataFrame
resultat = pd.DataFrame({'Start': last_date, 'End': end_date})
resultat


/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_4171/2328952231.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  end_date = df[df.index > last_date].groupby('tickers').apply(lambda x: x.index.max())


,Start,End
tickers,,
AAPL,2025-09-27,2025-10-01
ABNB,2025-09-27,2025-10-01
ADBE,2025-09-27,2025-10-01
ADI,2025-09-27,2025-10-01
ADP,2025-09-27,2025-10-01
...,...,...
VRTX,2025-09-27,2025-10-01
WBD,2025-09-27,2025-10-01
WDAY,2025-09-27,2025-10-01


In [13]:
"""
# Initialiser une DataFrame vide pour stocker toutes les données
data_SP500 = pd.DataFrame()

df = df.explode('tickers')
start_date = df.groupby('tickers').apply(lambda x: x.index.min())
end_date = df.groupby('tickers').apply(lambda x: x.index.max())

# Combiner les résultats dans une nouvelle DataFrame
resultat = pd.DataFrame({'Start': start_date, 'End': end_date})
resultat
"""

"\n# Initialiser une DataFrame vide pour stocker toutes les données\ndata_SP500 = pd.DataFrame()\n\ndf = df.explode('tickers')\nstart_date = df.groupby('tickers').apply(lambda x: x.index.min())\nend_date = df.groupby('tickers').apply(lambda x: x.index.max())\n\n# Combiner les résultats dans une nouvelle DataFrame\nresultat = pd.DataFrame({'Start': start_date, 'End': end_date})\nresultat\n"

In [14]:
# Télécharger les données pour chaque ticker
for ticker in resultat.index:
    start_date = resultat.loc[ticker, 'Start']
    end_date = resultat.loc[ticker, 'End']

    print(f"Téléchargement des données pour {ticker} de {start_date} à {end_date}")

    # Télécharger les données de l'action entre les dates spécifiées
    data = yf.download(
                        ticker,
                        start=start_date,
                        end=end_date,
                        interval="1d",
                        auto_adjust=True,
                        actions=False
                        )

    # Ajouter une colonne pour identifier le ticker
    data['Ticker'] = ticker

    # On retire le niveau des tickers pour ne garder que 'Close' et 'Volume'
    data.columns = data.columns.droplevel(level=1)
    
    # Concaténer les données dans la DataFrame globale
    data_SP500 = pd.concat([data_SP500, data])

# Affichage de la DataFrame résultante
data_SP500

Téléchargement des données pour AAPL de 2025-09-27 00:00:00 à 2025-10-01 00:00:00


[*********************100%***********************]  1 of 1 completed


Téléchargement des données pour ABNB de 2025-09-27 00:00:00 à 2025-10-01 00:00:00


[*********************100%***********************]  1 of 1 completed


Téléchargement des données pour ADBE de 2025-09-27 00:00:00 à 2025-10-01 00:00:00


...

Téléchargement des données pour XEL de 2025-09-27 00:00:00 à 2025-10-01 00:00:00


[*********************100%***********************]  1 of 1 completed


Téléchargement des données pour ZS de 2025-09-27 00:00:00 à 2025-10-01 00:00:00


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Ticker,Adj Close
Date,,,,,,,
2025-09-29,254.429993,255.000000,253.009995,254.559998,40127700.0,AAPL,NaN
2025-09-30,254.630005,255.919998,253.110001,254.860001,37666900.0,AAPL,NaN
2025-09-29,122.919998,124.580002,122.830002,124.440002,3874700.0,ABNB,NaN
2025-09-30,121.419998,123.080002,120.519997,122.900002,3728900.0,ABNB,NaN
2025-09-29,359.420013,363.170013,358.799988,362.040009,4075900.0,ADBE,NaN
...,...,...,...,...,...,...,...
2025-09-30,240.729996,249.139999,239.880005,248.460007,3001400.0,WDAY,NaN
2025-09-29,80.050003,80.070000,78.410004,79.459999,5527100.0,XEL,NaN
2025-09-30,80.650002,80.959999,79.589996,79.769997,4011000.0,XEL,NaN


In [15]:
data_SP500 = data_SP500.reset_index()
data_SP500

Price,Date,Close,High,Low,Open,Volume,Ticker,Adj Close
0,2025-09-29,254.429993,255.000000,253.009995,254.559998,40127700.0,AAPL,NaN
1,2025-09-30,254.630005,255.919998,253.110001,254.860001,37666900.0,AAPL,NaN
2,2025-09-29,122.919998,124.580002,122.830002,124.440002,3874700.0,ABNB,NaN
3,2025-09-30,121.419998,123.080002,120.519997,122.900002,3728900.0,ABNB,NaN
4,2025-09-29,359.420013,363.170013,358.799988,362.040009,4075900.0,ADBE,NaN
...,...,...,...,...,...,...,...,...
193,2025-09-30,240.729996,249.139999,239.880005,248.460007,3001400.0,WDAY,NaN
194,2025-09-29,80.050003,80.070000,78.410004,79.459999,5527100.0,XEL,NaN
195,2025-09-30,80.650002,80.959999,79.589996,79.769997,4011000.0,XEL,NaN
196,2025-09-29,296.899994,300.809998,293.510010,297.109985,1418600.0,ZS,NaN


In [16]:
df_concat = pd.concat([data_SP500, data_SP500_old])
df_concat

Price,Date,Close,High,Low,Open,Volume,Ticker,Adj Close
0,2025-09-29,254.429993,255.000000,253.009995,254.559998,40127700.0,AAPL,NaN
1,2025-09-30,254.630005,255.919998,253.110001,254.860001,37666900.0,AAPL,NaN
2,2025-09-29,122.919998,124.580002,122.830002,124.440002,3874700.0,ABNB,NaN
3,2025-09-30,121.419998,123.080002,120.519997,122.900002,3728900.0,ABNB,NaN
4,2025-09-29,359.420013,363.170013,358.799988,362.040009,4075900.0,ADBE,NaN
...,...,...,...,...,...,...,...,...
153,2025-09-26,177.570007,180.119995,174.910004,179.050003,44200200.0,PLTR,NaN
89,2025-09-26,208.190002,209.050003,207.350006,208.419998,3551600.0,HON,NaN
91,2025-09-26,632.219971,636.000000,615.330017,619.309998,344300.0,IDXX,NaN
93,2025-09-26,35.500000,36.299999,34.490002,35.020000,267991500.0,INTC,NaN


In [17]:
df_concat.sort_values(by=['Date'], ascending=True, inplace=True)
df_concat.tail(50)

Price,Date,Close,High,Low,Open,Volume,Ticker,Adj Close
105,2025-09-30,475.000000,478.779999,473.230011,473.700012,2171200.0,LIN,NaN
107,2025-09-30,133.899994,134.210007,130.809998,131.350006,10325700.0,LRCX,NaN
143,2025-09-30,203.619995,206.070007,201.919998,203.619995,4393500.0,PANW,NaN
109,2025-09-30,177.929993,180.089996,176.490005,178.529999,3412700.0,LULU,NaN
111,2025-09-30,260.440002,265.959991,259.619995,265.500000,1695300.0,MAR,NaN
141,2025-09-30,107.809998,108.720001,107.500000,107.500000,5027500.0,ORLY,NaN
113,2025-09-30,64.220001,64.870003,63.029999,64.269997,7685200.0,MCHP,NaN
115,2025-09-30,62.470001,62.869999,62.150002,62.450001,6603800.0,MDLZ,NaN
139,2025-09-30,49.310001,49.619999,47.900002,49.500000,8569500.0,ON,NaN
117,2025-09-30,2336.939941,2481.100098,2314.179932,2475.000000,928300.0,MELI,NaN


In [18]:
df_concat['Date'].max()

Timestamp('2025-09-30 00:00:00')

In [19]:
# Sauvegarder la DataFrame résultante dans un fichier CSV
#data_SP500.to_csv('/Users/forget/PyCharmMiscProject/historical_data_S&P500_From_Wiki.csv')

In [20]:
"""
mask_1 = data_SP500[(data_SP500['Ticker'] == 'NVR') & (data_SP500['Date'] < '1993-10-01')]
mask_2 = data_SP500[(data_SP500['Ticker'] == 'HUBB') & (data_SP500['Date'] < '1994-10-31')]
mask_3 = data_SP500[(data_SP500['Date'] == '1984-12-31') & (data_SP500['Ticker'] == 'BEN')]
"""

"\nmask_1 = data_SP500[(data_SP500['Ticker'] == 'NVR') & (data_SP500['Date'] < '1993-10-01')]\nmask_2 = data_SP500[(data_SP500['Ticker'] == 'HUBB') & (data_SP500['Date'] < '1994-10-31')]\nmask_3 = data_SP500[(data_SP500['Date'] == '1984-12-31') & (data_SP500['Ticker'] == 'BEN')]\n"

In [21]:
"""
idx_to_drop = mask_1.index.union(mask_2.index).union(mask_3.index)
data_SP500 = data_SP500.drop(index=idx_to_drop).copy()
"""

'\nidx_to_drop = mask_1.index.union(mask_2.index).union(mask_3.index)\ndata_SP500 = data_SP500.drop(index=idx_to_drop).copy()\n'

In [22]:
df_concat.to_parquet("/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet")